In [4]:
import neuraltda.topology2 as tp2
import pandas as pd
from joblib import Parallel, delayed
import neuraltda.simpComp as sc
import neuraltda.topology2 as tp2
import neuraltda.spectralAnalysis as sa
import datetime
import pickle
import glob
import os
from ephys import core, events, clust
import numpy as np
import tqdm
import matplotlib.pyplot as plt
import h5py as h5
%matplotlib inline
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

In [ ]:
# Rebin with 1ms, no overlap, start+2500ms to end+500ms, correct trials
block_path = ''
winsize = 1.0
dtover = 0.0
segment = [2500, 500]

cluster_group = ['Good', 'MUA']
widenarrow_threshold = 0.000230 # sw threshold in seconds

spikes = core.load_spikes(block_path)
trials = events.load_trials(block_path)
fs = core.load_fs(block_path)

# Get wide/narrow clusters
clusters = core.load_clusters(block_path)
clusters_list = clusters[clusters.quality.isin(cluster_group)]['cluster'].unique()
(wide, narrow) = clust.get_wide_narrow(block_path, clusters_list, widenarrow_threshold)

# Get Correct/Incorrect Trials
correctTrials = trials[trials['correct']==True]
incorrectTrials = trials[trials['correct']==False]

# Bin and 
bfdict = tp2.do_dag_bin_lazy(block_path, spikes, correctTrials, clusters, fs, winsize,
                                    segment, cluster_group=['Good', 'MUA'],
                                    dt_overlap=dtover, comment='SD-emily-reanalysis')
bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]
print('BDF: {}'.format(bdf))

In [ ]:
# contexts:
contexts = {
    'A_hi': 'T40S40D3',
    'A_lo': 'T3S3D40',
    'AS_hi': 'T40S40D-1',
    'AS_lo': 'T3S3D-1'
}


In [ ]:
# compute average psths
avg_psths = {}
with h5.File(bdf, 'r') as binned_data:
    for context in contexts.keys():
        stim = contexts[context]
        fulldata = np.array(binned_data[stim]['pop_tens'])
        avg_psth = np.mean(fulldata, axis=2)
        avg_psths[str(context)] = avg_psth
    binned_clusters = binned_data[stim]['clusters']

In [7]:
# Compute MSEs:

def mse(psth_a, psth_b, psth_c, psth_d):
    
    mse_ab = np.sum(np.square(psth_a - psth_b), axis=1)
    mse_cd = np.sum(np.square(psth_c - psth_d), axis=1)
    return mse_ab + mse_cd

MSE_targ = mse(avg_psths['A_hi'], avg_psths['AS_hi'], avg_psths['A_lo'], avg_psths['AS_lo'])
MSE_dist = mse(avg_psths['A_hi'], avg_psths['AS_lo'], avg_psths['A_lo'], avg_psths['AS_hi'])
MSE_diff = MSE_dist - MSE_targ

In [ ]:
# Break into wide/narrow
wide_mse_diff = MSE_diff[np.in1d(binned_clusters, wide)]
narrow_mse_diff = MSE_diff[np.in1d(binned_clusters, narrow)]

#Print wide
print('Wide Clusters')
for ind, val in enumerate(wide_mse_diff):
    print("Cluster: {}    MSE Difference: {:10.4f}".format(ind, val))
    
#Print Narrow
print('Narrow Clusters')
for ind, val in enumerate(narrow_mse_diff):
    print("Cluster: {}    MSE Difference: {:10.4f}".format(ind, val))

In [11]:
a_h = np.random.standard_normal((20, 100))
as_h = np.random.standard_normal((20, 100))
a_l = np.random.standard_normal((20, 100))
as_l = np.random.standard_normal((20, 100))

mse_t = mse(a_h, as_h, a_l, as_l)
mse_d = mse(a_h, as_l, a_l, as_h)
mse_diff = mse_d - mse_t

In [12]:
mse_diff

array([-20.25190463,  60.57024706, -38.27126792, -39.20822378,
        34.22506067, -21.56016953, -17.54711924,  56.44565946,
       -21.48444559,  -7.82825565,  89.79055453,  15.06173692,
        24.89982774, -17.02510418, -24.18807837,  52.66583291,
       -32.68713435,  48.00001554, -43.18510708,  11.15946077])